In [6]:
import pandas as pd

# Đọc 3 file 
df_basics = pd.read_csv('title.basics.tsv', sep='\t', low_memory=False)
df_ratings = pd.read_csv('title.ratings.tsv', sep='\t', low_memory=False)
df_crew = pd.read_csv('title.crew.tsv', sep='\t', low_memory=False)

In [7]:
# Tạo thêm thuộc tính primaryTitleLength bằng độ dài tên chính primaryTitle trong file title.basics.tsv --> thêm vào file title.basics
# Tạo thêm thuộc tính numGenres là số lượng thể loại được tính toán từ thuộc tính Genres trong file title.basics.tsv --> thêm vào file title.basics
# Tạo thêm thuộc tính writerCount là số lượng nhà văn được tính từ thuộc tính Writers trong file title.crew.tsv --> thêm vào file title.crew

In [8]:
# Đảm bảo tất cả các giá trị trong cột 'primaryTitle' là chuỗi và xử lý giá trị thiếu
df_basics['primaryTitle'] = df_basics['primaryTitle'].astype(str).fillna('')
# Tạo cột mới 'titleLength' chứa độ dài của mỗi giá trị trong cột 'primaryTitle'
df_basics['titleLength'] = df_basics['primaryTitle'].apply(lambda x: len(x))

In [9]:
# Tạo cột mới 'numGenres' là số lượng thể loại từ 'genres'
df_basics['numGenres'] = df_basics['genres'].apply(lambda x: len(x.split(',')) if pd.notnull(x) else 0)

In [10]:
print(df_basics)

             tconst  titleType               primaryTitle  \
0         tt0000001      short                 Carmencita   
1         tt0000002      short     Le clown et ses chiens   
2         tt0000003      short             Pauvre Pierrot   
3         tt0000004      short                Un bon bock   
4         tt0000005      short           Blacksmith Scene   
...             ...        ...                        ...   
10782261  tt9916848  tvEpisode              Episode #3.17   
10782262  tt9916850  tvEpisode              Episode #3.19   
10782263  tt9916852  tvEpisode              Episode #3.20   
10782264  tt9916856      short                   The Wind   
10782265  tt9916880  tvEpisode  Horrid Henry Knows It All   

                      originalTitle isAdult startYear endYear runtimeMinutes  \
0                        Carmencita       0      1894      \N              1   
1            Le clown et ses chiens       0      1892      \N              5   
2                    Pauvre

In [11]:
# Tạo thêm cột 'writerCount' là số lượng nhà văn từ cột 'writers'
df_crew['writerCount'] = df_crew['writers'].apply(lambda x: len(x.split(',')) if pd.notnull(x) else 0)

In [12]:
# Gộp 3 files thông qua thuộc tính 'tconst'
# Tạo ra file mới 'merged_df'
merged_df = df_basics.merge(df_ratings, on='tconst', how='inner')  # Inner join (keeps all rows)
merged_df = merged_df.merge(df_crew, on='tconst', how='inner')  # Merge with crew data

# Chuyển file thành định dạng csv
merged_df.to_csv('merged_data.tsv', sep='\t', index=False)  # Without index column

# In ra 
print(merged_df)

            tconst  titleType                primaryTitle  \
0        tt0000001      short                  Carmencita   
1        tt0000002      short      Le clown et ses chiens   
2        tt0000003      short              Pauvre Pierrot   
3        tt0000004      short                 Un bon bock   
4        tt0000005      short            Blacksmith Scene   
...            ...        ...                         ...   
1426275  tt9916730      movie                      6 Gunn   
1426276  tt9916766  tvEpisode              Episode #10.15   
1426277  tt9916778  tvEpisode                      Escape   
1426278  tt9916840  tvEpisode  Horrid Henry's Comic Caper   
1426279  tt9916880  tvEpisode   Horrid Henry Knows It All   

                      originalTitle isAdult startYear endYear runtimeMinutes  \
0                        Carmencita       0      1894      \N              1   
1            Le clown et ses chiens       0      1892      \N              5   
2                    Pauvre

In [13]:
# Đếm số lượng dòng có ít nhất 1 giá trị null
missing_row_count = merged_df.isnull().any(axis=1).sum()
print(missing_row_count)

3


In [14]:
# gán merged_df vào 1 dataframe khác và sử dụng hàm dropna
filtered_df = merged_df.dropna()
# gán lại dataframe vào merged_df ban đầu 
merged_df = filtered_df
# in ra số lượng dòng merged_df
print(len(merged_df))

1426277


In [15]:
import pandas as pd
import numpy as np
from scipy import stats

# Hàm tìm outliers sử dụng z-scores
def identify_outliers_zscore(df, threshold=3):
    # chọn ra các cột có kiểu dữ liệu là số
  numerical_cols = df.select_dtypes(include=np.number) 
    #khởi tạo 1 dataframe rỗng
  outliers_df = pd.DataFrame() 
    # duyệt qua từng cột
  for col in numerical_cols:
      # tính absolute z-scores
    z_scores = np.abs(stats.zscore(df[col])) 
      # Lấy dòng với outlier trong từng cột
    outlier_rows = df[z_scores > threshold].index  
      # Thêm outlier vào dataframe
    outliers_df = pd.concat([outliers_df, df.loc[outlier_rows]], ignore_index=True)  
  return outliers_df.drop_duplicates()  
    # Xoá trùng (duplicate) nếu tồn tại


In [16]:
outlier_row_df = identify_outliers_zscore(merged_df.copy(), 3)
print(outlier_row_df)

          tconst  titleType  \
0      tt0000053      short   
1      tt0000075      short   
2      tt0000166      short   
3      tt0000168      short   
4      tt0000206      short   
...          ...        ...   
59013  tt9878574  tvEpisode   
59014  tt9878576  tvEpisode   
59015  tt9881616      movie   
59016  tt9897038   tvSeries   
59017  tt9900092   tvSeries   

                                            primaryTitle  \
0      A Chegada do Comboio Inaugural à Estação Centr...   
1      The Conjuring of a Woman at the House of Rober...   
2      Salida de los trabajadores de la fábrica Españ...   
3      Salida del público de la iglesia parroquial de...   
4      Llegada de un tren a la estación de ferrocarri...   
...                                                  ...   
59013                                       Karamo Brown   
59014                                        Jacob Tobia   
59015                                    La filla d'algú   
59016                      

In [17]:
print(merged_df.shape)

(1426277, 16)


In [18]:
# Xoá những dòng vừa in ra (những dòng có ít nhất 1 giá trị bị outlier)
temp_df = merged_df.drop(outlier_row_df.index)  
# Gán lại temp_df cho merged_df
merged_df = temp_df
print(merged_df.shape)

(1368250, 16)


In [19]:

dup_row = merged_df[merged_df.duplicated(subset='primaryTitle', keep=False)]
print(dup_row)

            tconst  titleType          primaryTitle         originalTitle  \
26342    tt0044729      movie       Hurricane Smith       Hurricane Smith   
27019    tt0045555      movie          The Big Heat          The Big Heat   
27086    tt0045630      movie  City Beneath the Sea  City Beneath the Sea   
27142    tt0045698      movie    Angels of Darkness        Donne proibite   
27215    tt0045780      movie             Forbidden             Forbidden   
...            ...        ...                   ...                   ...   
1426249  tt9916362      movie                 Coven              Akelarre   
1426253  tt9916420  tvEpisode         Episode #21.1         Episode #21.1   
1426256  tt9916460    tvMovie             Pink Taxi             Pink Taxi   
1426276  tt9916766  tvEpisode        Episode #10.15        Episode #10.15   
1426277  tt9916778  tvEpisode                Escape                Escape   

        isAdult startYear endYear runtimeMinutes                       genr

In [20]:
print(merged_df.shape)

(1368250, 16)


In [21]:
merged_df_duplicates = merged_df[merged_df.duplicated(subset=['primaryTitle', 'originalTitle'])]
print(merged_df_duplicates.shape)

(326075, 16)


In [23]:
# Tạo merged_df_without_duplicates 
merged_df_without_duplicates = merged_df.drop_duplicates(subset=['primaryTitle', 'originalTitle'], keep = 'first')
# Gán merged_df_without_duplicates vào merged_df ban đầu
merged_df = merged_df_without_duplicates
# In ra thông số
print(merged_df.shape)

(1042175, 16)
